Ising model using Monte Carlo methods
=====================================



## Theoretical description



The [Ising model](https://en.wikipedia.org/wiki/Ising_model) is a statistical mechanics model originally developed to study [ferromagnetism](https://en.wikipedia.org/wiki/Ferromagnetism). A system
of magnetic particles can be described by a two dimensional arrangement (lattice) of atoms. To each atom
$i$ on the lattice, a discrete spin magnetic moment $\sigma_i=\pm 1$ is assigned.

<img src="../figures/ising_grid_small.png" alt="Schematic representation of the 2D spin arrangement" width="200"/>

The Hamiltonian of the system is given by:

<div class="LaTeX" id="orgd1901d9">
\begin{equation}
H = J\sum_{\langle ij \rangle} \sigma_i\sigma_j
\end{equation}

</div>

where the symbol $\langle ij \rangle$ implies a sum over the nearest neighbors and $J$ is the strength
of the exchange interaction. We will consider in this example ferromagnetic systems only, which implies
a value of $J > 0$.

The canonical [partition function](https://en.wikipedia.org/wiki/Partition_function_(statistical_mechanics)) of the system is given by:

<div class="LaTeX" id="org82178ff">
\begin{equation}
Z(\beta) = \sum_S e^{-\beta H}
\end{equation}

</div>

where $\beta = 1/k_B T$ with $T$ being the temperature and $k_B$ the Boltzmann’s constant. The sum runs
over all possible spin configurations $S$ of the system. Following the statistical mechanics reasoning, observables
are computed as follows:

<div class="LaTeX" id="org42b9c84">
\begin{equation}
\langle O \rangle = \frac{1}{Z(\beta)}\sum_S O(S) e^{-\beta H}
\end{equation}

</div>

The estimation of the observable's value is then given by:

<div class="LaTeX" id="org98c35d0">
\begin{equation}
\langle O \rangle \approx \frac{1}{N} \sum_i^{N} O_i 
\end{equation}

</div>

The total number of states for a square grid of length $L$ (**How many are there?**) suffers
from the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). In addition,
the exact solutions of the 2D Ising model are rather difficult to derive, see this [blog post](https://gandhiviswanathan.wordpress.com/2015/01/09/onsagers-solution-of-the-2-d-ising-model-the-combinatorial-method/) for a very nice sketch of the derivation. For these reasons, Monte Carlo methods are a great tool
to study this system.


### Metropolis algorithm



We will assume ergodicity in our implementation, together with the `single spin-flip dynamics` principle.
This means that in each transition we will change only one spin site on the lattice. We can then set our
transition probability to:

<div class="LaTeX" id="orgecc279e">
\begin{equation}
P(x, y) = g(x, y)A(x, y)
\end{equation}

</div>

where the selection probability is homogeneous $g(x, y) = 1/L^2$. The acceptance probability can be
set to the Metropolis choice after applying [detail balance](https://en.wikipedia.org/wiki/Detailed_balance):

<div class="LaTeX" id="orgffab8ae">
\begin{equation}
A(x, y) = 
  \begin{cases}
   e^{-\beta (H_x - H_y)} & H_x - H_y > 0 \\
   1 & \text{otherwise}
  \end{cases}
\end{equation}

</div>

The algorithm's steps are then:

1.  Pick a spin site with probability $g(x, y)$ and flip it
2.  Compute the energy change $\delta H=H_x-H_y$
3.  Accept the new configuration with probability $A(x, y)$
4.  Repeat



## Coding exercise



### Setting the environment



In [ ]:
"""Imports and global variables"""
import numpy as np
import matplotlib.pyplot as plt

### Main functions



In [ ]:
def init_state(L):
    """Initialized spin grid"""
    #TODO

def energ(grid, L):
    """Compute energy of new configuration"""
    en = 0
    for i in np.arange(L):
        for j in np.arange(L):
            s = grid[i, j]
            neighbor_s = grid[(i + 1) % L, j] + \
                         grid[(i - 1) % L, j] + \
                         grid[i, (j + 1) % L] + \
                         grid[i, (j - 1) % L]
            en -= s * neighbor_s
    return en / 2


def mc_update(grid, beta, L):
    """Updating step using the MH algorithm"""
    for _ in np.arange(L * L):
        # Select a center with probability 1/L^2
        # TODO
        
        # Get the spin value from selected center
        # TODO
        
        # Periodic boundary conditions (4 neighbors)
        neighbor_s = grid[(i + 1) % L, j] + \
                     grid[(i - 1) % L, j] + \
                     grid[i, (j + 1) % L] + \
                     grid[i, (j - 1) % L]
        
        # Change in energy
        dE = 2 * s * neighbor_s
        
        # Apply acceptance criterion
        # TODO
            
        # Update spin site
        # TODO
    
    return grid


### Set numerical experiment conditions



These are the input parameters of the simulation, they can be tuned in order to speed up or refine the results.
For convenience reasons, the temperature will be given in units of $[k_B/J]\cdot K$.



In [ ]:
# Parameter initialization (expensive settings, consider lowering)
NTEMP = 30
TEMP = np.linspace(1.5, 3.25, NTEMP)
LENGHT = 10
NSTEPS = 30000

# Variables initialization
ENERG = np.zeros(NTEMP)

### Run the simulation and compute expectation value



In [ ]:
for tdx, tval in enumerate(TEMP):
    # Compute beta factor
    BETA = 1 / tval

    # Initial grid
    GRID = init_state(LENGHT)

    # Run Monte Carlo steps
    ENT = 0
    # TODO

    # Set energy value
    ENERG[tdx] = ENT / (NSTEPS * LENGHT * LENGHT)

### Find critical temperature



In [ ]:
# PLot curve
plt.plot(TEMP, ENERG, 'o-')

# Inflection point
# TODO

# Show
# TODO (add labels with correct units)
plt.show()

### Questions:



1.  Complete all missing code sections
2.  Plot the energy temperature curve
3.  Which are the units of the energy and temperature on the plot?
4.  What is the value of the critical temperature? Compare with the [analytical](https://en.wikipedia.org/wiki/Square_lattice_Ising_model#Critical_temperature) value.

